In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

FHIR_DIR = "/content/drive/MyDrive/Patient_records/fhir (FHIR R4 Synthea)"
print("FHIR bundles will be read from:", FHIR_DIR)

Mounted at /content/drive
FHIR bundles will be read from: /content/drive/MyDrive/Patient_records/fhir (FHIR R4 Synthea)


In [2]:
%%bash
# (re‑)create venv
python -m pip install --quiet virtualenv
python -m virtualenv /content/fhir_env

# activate & upgrade pip
source /content/fhir_env/bin/activate
pip install --quiet --upgrade pip

# core libs (NumPy<2 for compatibility, PyTorch 2.3 CUDA 12, etc.)
pip install --quiet "numpy<2" \
    --extra-index-url https://download.pytorch.org/whl/cu121 \
    torch==2.3.0 torchvision==0.18.0

# helper libs for the clinical assistant
pip install --quiet \
    fhir.resources==7.1.0 \
    sentence-transformers==2.7.0 \
    transformers==4.43.2 accelerate==0.30.1 bitsandbytes==0.43.0 \
    langchain==0.2.0 langchain-community==0.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 38.0 MB/s eta 0:00:00
created virtual environment CPython3.11.13.final.0-64 in 339ms
  creator CPython3Posix(dest=/content/fhir_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.1.1, setuptools==80.3.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 141.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 133.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 136.7 MB/s et

In [3]:
%%bash
rm -rf /content/faiss
rm -rf /content/fhir_env/lib/python3.11/site-packages/faiss*
rm -rf /content/fhir_env/lib/python3.11/site-packages/_swigfaiss*

In [4]:
%%bash
source /content/fhir_env/bin/activate
pip install --upgrade --force-reinstall "numpy<2"

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


In [ ]:
%%bash
source /content/fhir_env/bin/activate
pip install --quiet faiss-cpu


In [ ]:
import faiss
print("faiss imported", faiss.__version__)


In [ ]:
import faiss, torch, numpy as np
d = 32
cpu_index = faiss.IndexFlatL2(d)
res       = faiss.StandardGpuResources()
gpu_index = faiss.index_cpu_to_gpu(res, 0, cpu_index)

xb = np.random.randn(10_000, d).astype('float32')
gpu_index.add(xb)
D, _ = gpu_index.search(xb[:5], 3)
print("✓ GPU Faiss operational – distances shape:", D.shape)

print("CUDA visible to Torch?:", torch.cuda.is_available(),
      "| device name:", torch.cuda.get_device_name(0))